# Importing libraries

In [1]:
import os, glob, json, csv, subprocess, sys, re
from git import *
from subprocess import Popen, PIPE
from os import path
import pandas as pd
import itertools

# Defining repository

In [2]:
userhome = os.path.expanduser("~")
repository = userhome + r"/historage-mining/historage/hbase/"

# Defining function to execute git command

In [3]:
def execute_command(cmd, work_dir):
    #Executes a shell command in a subprocess, waiting until it has completed.
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

# Finding the history of files

In [4]:
colnames = ['bug_ref','commitID_1','commitID_2','filename','#line_changed-hist','#line_changed-myers','#line_changed-min','#line_changed-pat']
#Show the csv file with different value of number of line changed
data = pd.read_csv('diffbugs_file_differentvalue_withfilepath.csv')
data = data[colnames]
data

,bug_ref,commitID_1,commitID_2,filename,#line_changed-hist,#line_changed-myers,#line_changed-min,#line_changed-pat
0,HBASE-10569,0e5d21e96ae24e8c658a1b74843b72bddcb7435,7716a3ca94,/Users/YusufNugroho/historage-mining/historage...,31,29,29,31
1,HBASE-13686,587b0b4f20bdc0415b6541023e611b69c87dba15,9f43a3bea6,/Users/YusufNugroho/historage-mining/historage...,103,97,97,99
2,HBASE-13686,587b0b4f20bdc0415b6541023e611b69c87dba15,9f43a3bea6,/Users/YusufNugroho/historage-mining/historage...,114,108,108,108
3,HBASE-10569,0e5d21e96ae24e8c658a1b74843b72bddcb7435,7716a3ca94,/Users/YusufNugroho/historage-mining/historage...,1993,1977,1969,1985
4,HBASE-10569,0e5d21e96ae24e8c658a1b74843b72bddcb7435,7716a3ca94,/Users/YusufNugroho/historage-mining/historage...,2285,2425,2285,2285


In [5]:
filename = []
for jj in range(0,len(data)):
    filename.append(((data.iloc[jj][3].split('/'))[-1:])[0])
filename

['ActiveMasterManager.java',
 'RateLimiter.java',
 'TestRateLimiter.java',
 'HMaster.java',
 'HRegionServer.java']

# Applying "git blame" for files

In [6]:
pattern1 = re.compile(r'(?P<commit_id>[\^]\w+|\w+)\s+(?P<filename>[^\s]+)\s+\((?P<committer>.*?)\s+(?P<date>\d{4}-\d\d-\d\d)\s+(?P<time>\d\d:\d\d:\d\d).*?(?P<line_number>\b\d+\b)\)\s(?P<code>[^`]*)')
pattern2 = re.compile(r'(?P<commit_id>[\^]\w+|\w+)\s+\((?P<committer>.*?)\s+(?P<date>\d{4}-\d\d-\d\d)\s+(?P<time>\d\d:\d\d:\d\d).*?(?P<line_number>\b\d+\b)\)\s(?P<code>[^`]*)')

# Applying "git blame" for ActiveMasterManager.java

In [22]:
parent = data.iloc[0][1]
file = data.iloc[0][3]

#checkout parentID and applying git blame
checkout = "git checkout " + parent
blame = "git blame " + file

execute_command(checkout, repository)
result = (str(execute_command(blame, repository)).replace("b'",''))
result = re.split(r'(?<!\\)\\n', result)
del result[len(result)-1]
temp = []
if result[0][0] == "(":
    temp.append(result[0][1:])
    
for row in result[1:]:
    temp.append(row)

#exit from checkout
excheck = "git checkout master"
execute_command(excheck, repository)

print("git blame has been executed")

git blame has been executed


In [23]:
with open('/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/step_3/blame_' + filename[0] + '.csv', 'w+', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['commit_id', 'filename', 'committer', 'date', 'time', 'line_number', 'code'])
    for line in temp:
        writer.writerow(pattern1.match(line).groups())
        #print(pattern1.match(line).groups())

In [24]:
dfblame = pd.read_csv('/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/step_3/blame_' + filename[0] + '.csv')
dfblame.head()

,commit_id,filename,committer,date,time,line_number,code
0,f5213095324,src/main/java/org/apache/hadoop/hbase/master/A...,Michael Stack,2010-08-31,23:51:44,1,/**
1,f5213095324,src/main/java/org/apache/hadoop/hbase/master/A...,Michael Stack,2010-08-31,23:51:44,2,*
2,f5213095324,src/main/java/org/apache/hadoop/hbase/master/A...,Michael Stack,2010-08-31,23:51:44,3,* Licensed to the Apache Software Foundation ...
3,f5213095324,src/main/java/org/apache/hadoop/hbase/master/A...,Michael Stack,2010-08-31,23:51:44,4,* or more contributor license agreements. Se...
4,f5213095324,src/main/java/org/apache/hadoop/hbase/master/A...,Michael Stack,2010-08-31,23:51:44,5,* distributed with this work for additional i...


# Applying "git blame" for RateLimiter.java

In [10]:
parent = data.iloc[1][1]
file = data.iloc[1][3]

#checkout parentID and applying git blame
checkout = "git checkout " + parent
blame = "git blame " + file

execute_command(checkout, repository)
result = (str(execute_command(blame, repository)).replace("b'",''))
result = re.split(r'(?<!\\)\\n', result)
del result[len(result)-1]
temp = []
if result[0][0] == "(":
    temp.append(result[0][1:])
    
for row in result[1:]:
    temp.append(row)

#exit from checkout
excheck = "git checkout master"
execute_command(excheck, repository)

print("git blame has been executed")

git blame has been executed


In [11]:
with open('/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/step_3/blame_' + filename[1] + '.csv', 'w+', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['commit_id', 'committer', 'date', 'time', 'line_number', 'code'])
    for line in temp:
        writer.writerow(pattern2.match(line).groups())
        #print (pattern2.match(line).groups())

In [12]:
dfblame = pd.read_csv('/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/step_3/blame_' + filename[1] + '.csv')
dfblame.head()

,commit_id,committer,date,time,line_number,code
0,bd8df9ccd4d,Matteo Bertozzi,2014-09-18,01:51:15,1,/**
1,bd8df9ccd4d,Matteo Bertozzi,2014-09-18,01:51:15,2,* Licensed to the Apache Software Foundation ...
2,bd8df9ccd4d,Matteo Bertozzi,2014-09-18,01:51:15,3,* or more contributor license agreements. Se...
3,bd8df9ccd4d,Matteo Bertozzi,2014-09-18,01:51:15,4,* distributed with this work for additional i...
4,bd8df9ccd4d,Matteo Bertozzi,2014-09-18,01:51:15,5,* regarding copyright ownership. The ASF lic...


# Applying "git blame" for TestRateLimiter.java

In [13]:
parent = data.iloc[2][1]
file = data.iloc[2][3]

#checkout parentID and applying git blame
checkout = "git checkout " + parent
blame = "git blame " + file

execute_command(checkout, repository)
result = (str(execute_command(blame, repository)).replace("b'",''))
result = re.split(r'(?<!\\)\\n', result)
del result[len(result)-1]
temp = []
if result[0][0] == "(":
    temp.append(result[0][1:])
    
for row in result[1:]:
    temp.append(row)

#exit from checkout
excheck = "git checkout master"
execute_command(excheck, repository)

print("git blame has been executed")

git blame has been executed


In [14]:
with open('/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/step_3/blame_' + filename[2] + '.csv', 'w+', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['commit_id', 'committer', 'date', 'time', 'line_number', 'code'])
    for line in temp:
        writer.writerow(pattern2.match(line).groups())

In [15]:
dfblame = pd.read_csv('/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/step_3/blame_' + filename[2] + '.csv')
dfblame.head()

,commit_id,committer,date,time,line_number,code
0,bd8df9ccd4d,Matteo Bertozzi,2014-09-18,01:51:15,1,/**
1,bd8df9ccd4d,Matteo Bertozzi,2014-09-18,01:51:15,2,* Licensed to the Apache Software Foundation ...
2,bd8df9ccd4d,Matteo Bertozzi,2014-09-18,01:51:15,3,* or more contributor license agreements. Se...
3,bd8df9ccd4d,Matteo Bertozzi,2014-09-18,01:51:15,4,* distributed with this work for additional i...
4,bd8df9ccd4d,Matteo Bertozzi,2014-09-18,01:51:15,5,* regarding copyright ownership. The ASF lic...


# Applying "git blame" for HMaster.java

In [16]:
parent = data.iloc[3][1]
file = data.iloc[3][3]

#checkout parentID and applying git blame
checkout = "git checkout " + parent
blame = "git blame " + file

execute_command(checkout, repository)
result = (str(execute_command(blame, repository)).replace("b'",''))
result = re.split(r'(?<!\\)\\n', result)
del result[len(result)-1]
temp = []
if result[0][0] == "(":
    temp.append(result[0][1:])
    
for row in result[1:]:
    temp.append(row)

#exit from checkout
excheck = "git checkout master"
execute_command(excheck, repository)

print("git blame has been executed")

git blame has been executed


In [17]:
with open('/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/step_3/blame_' + filename[3] + '.csv', 'w+', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['commit_id', 'filename', 'committer', 'date', 'time', 'line_number', 'code'])
    for line in temp:
        writer.writerow(pattern1.match(line).groups())
        #print (pattern.match(line).groups())

In [18]:
dfblame = pd.read_csv('/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/step_3/blame_' + filename[3] + '.csv')
dfblame.head()

,commit_id,filename,committer,date,time,line_number,code
0,^948d0cdec4,src/java/org/apache/hadoop/hbase/HMaster.java,Doug Cutting,2007-04-03,20:34:28,1,/**
1,^948d0cdec4,src/java/org/apache/hadoop/hbase/HMaster.java,Doug Cutting,2007-04-03,20:34:28,2,*
2,c6bf382ebe9,src/java/org/apache/hadoop/hbase/HMaster.java,Jim Kellerman,2007-07-12,22:08:25,3,* Licensed to the Apache Software Foundation ...
3,c6bf382ebe9,src/java/org/apache/hadoop/hbase/HMaster.java,Jim Kellerman,2007-07-12,22:08:25,4,* or more contributor license agreements. Se...
4,c6bf382ebe9,src/java/org/apache/hadoop/hbase/HMaster.java,Jim Kellerman,2007-07-12,22:08:25,5,* distributed with this work for additional i...


# Applying "git blame" for HRegionServer.java

In [19]:
parent = data.iloc[4][1]
file = data.iloc[4][3]

#checkout parentID and applying git blame
checkout = "git checkout " + parent
blame = "git blame " + file

execute_command(checkout, repository)
result = (str(execute_command(blame, repository)).replace("b'",''))
result = re.split(r'(?<!\\)\\n', result)
del result[len(result)-1]
temp = []
if result[0][0] == "(":
    temp.append(result[0][1:])
    
for row in result[1:]:
    temp.append(row)

#exit from checkout
excheck = "git checkout master"
execute_command(excheck, repository)

print("git blame has been executed")

git blame has been executed


In [20]:
with open('/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/step_3/blame_' + filename[4] + '.csv', 'w+', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['commit_id', 'filename', 'committer', 'date', 'time', 'line_number', 'code'])
    for line in temp:
        writer.writerow(pattern1.match(line).groups())

In [25]:
dfblame = pd.read_csv('/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/step_3/blame_' + filename[4] + '.csv')
dfblame.head()

,commit_id,filename,committer,date,time,line_number,code
0,8763e9e92c4,src/java/org/apache/hadoop/hbase/regionserver/...,Jim Kellerman,2008-08-11,17:57:53,1,/**
1,8763e9e92c4,src/java/org/apache/hadoop/hbase/regionserver/...,Jim Kellerman,2008-08-11,17:57:53,2,*
2,8763e9e92c4,src/java/org/apache/hadoop/hbase/regionserver/...,Jim Kellerman,2008-08-11,17:57:53,3,* Licensed to the Apache Software Foundation ...
3,8763e9e92c4,src/java/org/apache/hadoop/hbase/regionserver/...,Jim Kellerman,2008-08-11,17:57:53,4,* or more contributor license agreements. Se...
4,8763e9e92c4,src/java/org/apache/hadoop/hbase/regionserver/...,Jim Kellerman,2008-08-11,17:57:53,5,* distributed with this work for additional i...
